In [3]:
# Запись CUDA-кода в файл stack_cuda.cu
%%writefile part1.cu

// Подключение CUDA Runtime (память, ядра, атомарные операции)
#include <cuda_runtime.h>

// Подключение стандартного вывода
#include <iostream>

// Максимальная вместимость стека
#define STACK_CAPACITY 1024

// Количество потоков CUDA
#define THREADS 256

//
// Структура параллельного стека
//

// Стек, размещённый в глобальной памяти GPU
struct Stack {
    int data[STACK_CAPACITY]; // Массив элементов стека
    int top;                  // Индекс вершины стека
};

//
// Добавление элемента в стек
//

// Устройство-функция для операции push
__device__ bool stack_push(Stack* stack, int value) {

    // Атомарно увеличиваем указатель вершины
    int position = atomicAdd(&stack->top, 1);

    // Проверка на переполнение стека
    if (position >= STACK_CAPACITY) {
        // Откат изменения, если стек переполнен
        atomicSub(&stack->top, 1);
        return false;
    }

    // Записываем значение в стек
    stack->data[position] = value;
    return true;
}

//
// Удаление элемента из стека
//

// Устройство-функция для операции pop
__device__ bool stack_pop(Stack* stack, int* result) {

    // Атомарно уменьшаем вершину стека
    int position = atomicSub(&stack->top, 1) - 1;

    // Проверка на пустой стек
    if (position < 0) {
        // Возвращаем указатель вершины назад
        atomicAdd(&stack->top, 1);
        return false;
    }

    // Считываем значение
    *result = stack->data[position];
    return true;
}

//
// Ядро: параллельное добавление
//

// Каждый поток кладёт значение в стек
__global__ void push_kernel(Stack* stack) {

    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    stack_push(stack, tid);
}

//
// Ядро: параллельное удаление
//

// Каждый поток пытается извлечь значение из стека
__global__ void pop_kernel(Stack* stack, int* output) {

    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int value;

    if (stack_pop(stack, &value))
        output[tid] = value;
    else
        output[tid] = -1;
}

//
// Главная функция
//

int main() {

    // Указатели на данные в памяти GPU
    Stack* d_stack;
    int* d_output;

    // Выделение памяти на GPU
    cudaMalloc(&d_stack, sizeof(Stack));
    cudaMalloc(&d_output, THREADS * sizeof(int));

    // Создание и инициализация стека на CPU
    Stack h_stack;
    h_stack.top = 0;

    // Копирование стека на GPU
    cudaMemcpy(d_stack, &h_stack, sizeof(Stack), cudaMemcpyHostToDevice);

    // Запуск ядра добавления элементов
    push_kernel<<<1, THREADS>>>(d_stack);
    cudaDeviceSynchronize();

    // Запуск ядра извлечения элементов
    pop_kernel<<<1, THREADS>>>(d_stack, d_output);
    cudaDeviceSynchronize();

    // Массив для хранения результатов на CPU
    int h_output[THREADS];

    // Копирование результатов с GPU
    cudaMemcpy(h_output, d_output, THREADS * sizeof(int), cudaMemcpyDeviceToHost);

    // Подсчёт успешных операций pop
    int successful = 0;
    for (int i = 0; i < THREADS; i++) {
        if (h_output[i] != -1)
            successful++;
    }

    // Вывод результатов
    std::cout << "Количество успешных операций pop: " << successful << std::endl;
    std::cout << "Максимально возможное значение: " << STACK_CAPACITY << std::endl;

    // Освобождение памяти GPU
    cudaFree(d_stack);
    cudaFree(d_output);

    return 0;
}


Overwriting part1.cu


In [4]:
!nvcc part1.cu -o part1

In [5]:

!./part1

Количество успешных операций pop: 256
Максимально возможное значение: 1024


In [7]:
# Запись CUDA-кода в файл queue_vs_stack.cu
%%writefile part2.cu

// Подключение CUDA Runtime (память, ядра, атомарные операции)
#include <cuda_runtime.h>

// Подключение стандартного вывода
#include <iostream>

// Максимальная вместимость стека и очереди
#define CAPACITY 1024

// Количество CUDA-потоков
#define THREADS 256

//
// Структура параллельного стека
//

// Стек (LIFO), размещённый в глобальной памяти GPU
struct Stack {
    int data[CAPACITY]; // Массив для хранения элементов
    int top;            // Указатель вершины стека
};

//
// Структура параллельной очереди
//

// Очередь (FIFO), реализованная как кольцевой буфер
struct Queue {
    int data[CAPACITY]; // Буфер для элементов
    int head;           // Индекс чтения
    int tail;           // Индекс записи
    int size;           // Текущее количество элементов
};

//
// Операция push для стека
//

// Устройство-функция добавления элемента в стек
__device__ bool stack_push(Stack* stack, int value) {

    // Атомарно увеличиваем указатель вершины
    int pos = atomicAdd(&stack->top, 1);

    // Проверка переполнения
    if (pos >= CAPACITY) {
        atomicSub(&stack->top, 1);
        return false;
    }

    // Запись значения
    stack->data[pos] = value;
    return true;
}

//
// Операция pop для стека
//

// Устройство-функция извлечения элемента из стека
__device__ bool stack_pop(Stack* stack, int* result) {

    // Атомарно уменьшаем указатель вершины
    int pos = atomicSub(&stack->top, 1) - 1;

    // Проверка на пустой стек
    if (pos < 0) {
        atomicAdd(&stack->top, 1);
        return false;
    }

    // Считывание значения
    *result = stack->data[pos];
    return true;
}

//
// Операция enqueue для очереди
//

// Устройство-функция добавления элемента в очередь
__device__ bool queue_enqueue(Queue* queue, int value) {

    // Резервируем позицию для записи
    int pos = atomicAdd(&queue->tail, 1);

    // Увеличиваем размер очереди и проверяем переполнение
    if (atomicAdd(&queue->size, 1) >= CAPACITY) {
        atomicSub(&queue->tail, 1);
        atomicSub(&queue->size, 1);
        return false;
    }

    // Запись элемента (кольцевой буфер)
    queue->data[pos % CAPACITY] = value;
    return true;
}

//
// Операция dequeue для очереди
//

// Устройство-функция извлечения элемента из очереди
__device__ bool queue_dequeue(Queue* queue, int* result) {

    // Уменьшаем размер очереди и проверяем пустоту
    if (atomicSub(&queue->size, 1) <= 0) {
        atomicAdd(&queue->size, 1);
        return false;
    }

    // Резервируем позицию для чтения
    int pos = atomicAdd(&queue->head, 1);

    // Считывание элемента
    *result = queue->data[pos % CAPACITY];
    return true;
}

//
// CUDA-ядра для стека
//

// Параллельное добавление в стек
__global__ void stack_push_kernel(Stack* stack) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    stack_push(stack, tid);
}

// Параллельное извлечение из стека
__global__ void stack_pop_kernel(Stack* stack, int* output) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int value;

    if (stack_pop(stack, &value))
        output[tid] = value;
    else
        output[tid] = -1;
}

//
// CUDA-ядра для очереди
//

// Параллельное добавление в очередь
__global__ void queue_enqueue_kernel(Queue* queue) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    queue_enqueue(queue, tid);
}

// Параллельное извлечение из очереди
__global__ void queue_dequeue_kernel(Queue* queue, int* output) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int value;

    if (queue_dequeue(queue, &value))
        output[tid] = value;
    else
        output[tid] = -1;
}

//
// Главная функция
//

int main() {

    // Указатели на данные в памяти GPU
    Stack* d_stack;
    Queue* d_queue;
    int* d_output;

    // Выделение памяти на GPU
    cudaMalloc(&d_stack, sizeof(Stack));
    cudaMalloc(&d_queue, sizeof(Queue));
    cudaMalloc(&d_output, THREADS * sizeof(int));

    // Инициализация стека на CPU
    Stack h_stack;
    h_stack.top = 0;

    // Инициализация очереди на CPU
    Queue h_queue;
    h_queue.head = 0;
    h_queue.tail = 0;
    h_queue.size = 0;

    // Копирование данных на GPU
    cudaMemcpy(d_stack, &h_stack, sizeof(Stack), cudaMemcpyHostToDevice);
    cudaMemcpy(d_queue, &h_queue, sizeof(Queue), cudaMemcpyHostToDevice);

    // События CUDA для измерения времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    //
    // Измерение времени стека
    //

    cudaEventRecord(start);

    stack_push_kernel<<<1, THREADS>>>(d_stack);
    stack_pop_kernel<<<1, THREADS>>>(d_stack, d_output);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float stack_time;
    cudaEventElapsedTime(&stack_time, start, stop);

    //
    // Измерение времени очереди
    //

    cudaEventRecord(start);

    queue_enqueue_kernel<<<1, THREADS>>>(d_queue);
    queue_dequeue_kernel<<<1, THREADS>>>(d_queue, d_output);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float queue_time;
    cudaEventElapsedTime(&queue_time, start, stop);

    //
    // Вывод результатов
    //

    std::cout << "Время выполнения стека: " << stack_time << " мс" << std::endl;
    std::cout << "Время выполнения очереди: " << queue_time << " мс" << std::endl;

    // Освобождение памяти GPU
    cudaFree(d_stack);
    cudaFree(d_queue);
    cudaFree(d_output);

    return 0;
}



Writing part2.cu


In [8]:

!nvcc part2.cu -o part2

In [9]:
!./part2

Время выполнения стека: 11.2188 мс
Время выполнения очереди: 0.002528 мс
